In [1]:
from pyspark.sql import Window
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

ModuleNotFoundError: No module named 'numpy'

In [ ]:
# =============================================
# Spark Session với HDFS Configuration
# =============================================
spark = SparkSession.builder \
    .appName("MovieLensALS") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

# Đặt log level để giảm noise
spark.sparkContext.setLogLevel("WARN")

25/11/13 17:52:18 WARN Utils: Your hostname, DESKTOP-8DIE4OP resolves to a loopback address: 127.0.1.1; using 172.20.177.130 instead (on interface eth0)
25/11/13 17:52:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/13 17:52:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
import time

# Thử tạo session mới
try:
    spark = SparkSession.builder \
        .appName("ML-32M-Optimized") \
        .config("spark.driver.memory", "4g") \
        .config("spark.executor.memory", "2g") \
        .config("spark.driver.maxResultSize", "2g") \
        .config("spark.memory.fraction", "0.8") \
        .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
        .getOrCreate()
    
    print("✅ Đã tạo Spark session mới với cấu hình tối ưu")
    
except Exception as e:
    print(f"Lỗi khi tạo session: {e}")
    print("Thử tạo session với config đơn giản hơn...")
    
    spark = SparkSession.builder \
        .appName("ML-32M-Simple") \
        .config("spark.driver.memory", "2g") \
        .getOrCreate()
    
    print("✅ Đã tạo Spark session với config đơn giản")

25/11/13 22:41:32 WARN Utils: Your hostname, DESKTOP-8DIE4OP resolves to a loopback address: 127.0.1.1; using 172.20.177.130 instead (on interface eth0)
25/11/13 22:41:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/13 22:41:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


✅ Đã tạo Spark session mới với cấu hình tối ưu


In [ ]:
# Dataset 32M
ratings_path = "/movielens/32M/ratings.csv"
movies_path = "/movielens/32M/movies.csv"

ratings_df = spark.read.csv(ratings_path, header=True, inferSchema=True)
movies_df = spark.read.csv(movies_path, header=True, inferSchema=True)

ratings_df.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|     17|   4.0|944249077|
|     1|     25|   1.0|944250228|
|     1|     29|   2.0|943230976|
|     1|     30|   5.0|944249077|
|     1|     32|   5.0|943228858|
+------+-------+------+---------+
only showing top 5 rows



In [5]:
from pyspark.sql import Window
from pyspark.sql import functions as F
# Chuẩn hóa dữ liệu cho ALS
print("\n=== Chuẩn hóa dữ liệu ===")
als_data = ratings_df.select(
    F.col("userId").cast("integer").alias("userId"),
    F.col("movieId").cast("integer").alias("movieId"),
    F.col("rating").cast("float").alias("rating")
).dropna()


print(f"Dữ liệu sau chuẩn hóa: {als_data.count():,}")
als_data.show(5)


=== Chuẩn hóa dữ liệu ===


Dữ liệu sau chuẩn hóa: 32,000,204
+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|     17|   4.0|
|     1|     25|   1.0|
|     1|     29|   2.0|
|     1|     30|   5.0|
|     1|     32|   5.0|
+------+-------+------+
only showing top 5 rows



In [ ]:
print("=== Lọc dữ liệu sparse ===")
user_counts = als_data.groupBy("userId").count()
item_counts = als_data.groupBy("movieId").count()

# Chỉ giữ lại users có ít nhất 5 ratings và items có ít nhất 10 ratings
min_user_ratings = 5
min_item_ratings = 10

filtered_als_data = als_data \
    .join(user_counts.filter(F.col("count") >= min_user_ratings), "userId") \
    .join(item_counts.filter(F.col("count") >= min_item_ratings), "movieId") \
    .select("userId", "movieId", "rating")

print(f"Dữ liệu sau khi lọc sparse: {filtered_als_data.count():,}")
print(f"Số users duy nhất: {filtered_als_data.select('userId').distinct().count():,}")
print(f"Số movies duy nhất: {filtered_als_data.select('movieId').distinct().count():,}")


=== Lọc dữ liệu sparse ===


Dữ liệu sau khi lọc sparse: 31,842,705


Số users duy nhất: 200,948


[Stage 45:=======>          (3 + 4) / 7][Stage 46:>                 (0 + 0) / 7]

In [ ]:
print("\n=== Chia train/test ===")
w = Window.partitionBy("userId").orderBy(F.rand(seed=42))
ranked = filtered_als_data.withColumn("rank", F.row_number().over(w))

test_data = ranked.filter(F.col("rank") == 1).drop("rank")
train_data = ranked.filter(F.col("rank") > 1).drop("rank")

In [12]:
als = ALS(
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating", 
    coldStartStrategy="drop",
    nonnegative=True,
    rank=10,           # Giảm rank
    maxIter=5,         # Giảm iterations
    regParam=0.1,
    numUserBlocks=50,  # Tăng số blocks
    numItemBlocks=50
)

# Train với checkpointing
spark.sparkContext.setCheckpointDir("/tmp/checkpoints")
model_32m = als.fit(train_data)


=== Training ALS model ===


[Stage 105:==============================================>        (42 + 4) / 50]

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
model_path = "hdfs://localhost:9000/user/haanh/movielens/models/als_32M"
model.write().overwrite().save(model_path)
print(f"💾 Mô hình ALS đã được lưu tại: {model_path}")

In [6]:
als_data = ratings_sampled.select(
    F.col("userId").cast("integer").alias("userId"),
    F.col("movieId").cast("integer").alias("movieId"),
    F.col("rating").cast("float").alias("rating")
).dropna()

NameError: name 'F' is not defined

In [3]:
# Chuẩn hóa cho ALS
als_data = ratings_df.select(
    col("userId"),
    col("movieId"),
    col("rating").cast("float")
).dropna()

print("Total ratings:", als_data.count())



[Stage 3:==================================================>        (6 + 1) / 7]

Total ratings: 32000204


In [4]:
from pyspark.sql import Window
from pyspark.sql import functions as F

w = Window.partitionBy("userId").orderBy(F.rand())
ranked = als_data.withColumn("rank", F.row_number().over(w))
test_data = ranked.filter(F.col("rank") == 1).drop("rank")
train_data = ranked.filter(F.col("rank") > 1).drop("rank")

print("Train:", train_data.count(), " Test:", test_data.count())

[Stage 12:=================================================>        (6 + 1) / 7]

Train: 31799256  Test: 200948


In [5]:
print("Train size:", train_data.count())
print("Test size:", test_data.count())
print("Số user trong train:", train_data.select("userId").distinct().count())
print("Số user trong test:", test_data.select("userId").distinct().count())

Train size: 31799256


Test size: 200948


Số user trong train: 200948


[Stage 36:=================================================>        (6 + 1) / 7]

Số user trong test: 200948


In [5]:
als = ALS(
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating", 
    coldStartStrategy="drop",
    nonnegative=True,
    rank=20,           # Giảm rank
    maxIter=5,         # Giảm iterations
    regParam=0.1,
    numUserBlocks=50,  # Tăng số blocks
    numItemBlocks=50
)

# Train với checkpointing
spark.sparkContext.setCheckpointDir("/tmp/checkpoints")
model_32m = als.fit(train_data)

=== Training ALS model ===


25/11/13 15:15:46 WARN MemoryStore: Not enough space to cache rdd_124_8 in memory! (computed 3.3 MiB so far)
25/11/13 15:15:46 WARN BlockManager: Persisting block rdd_124_8 to disk instead.
25/11/13 15:16:08 WARN BlockManager: Block rdd_127_5 could not be removed as it was not found on disk or in memory
25/11/13 15:16:08 WARN BlockManager: Block rdd_128_5 could not be removed as it was not found on disk or in memory
25/11/13 15:16:09 ERROR Executor: Exception in task 5.0 in stage 49.0 (TID 127)
java.lang.OutOfMemoryError: Java heap space
	at java.util.Arrays.copyOf(Arrays.java:3284)
	at scala.collection.mutable.ArrayBuilder$ofInt.mkArray(ArrayBuilder.scala:339)
	at scala.collection.mutable.ArrayBuilder$ofInt.result(ArrayBuilder.scala:383)
	at scala.collection.mutable.ArrayBuilder$ofInt.result(ArrayBuilder.scala:330)
	at org.apache.spark.ml.recommendation.ALS$UncompressedInBlockBuilder.build(ALS.scala:1452)
	at org.apache.spark.ml.recommendation.ALS$.$anonfun$makeBlocks$5(ALS.scala:1660

ConnectionRefusedError: [Errno 111] Connection refused

ConnectionRefusedError: [Errno 111] Connection refused

In [3]:
# Sửa đường dẫn thành file .dat
ratings_path = "hdfs://localhost:9000/user/haanh/movielens/1M/ratings.dat"
ratings_df = spark.read.csv(ratings_path, sep='::', header=False, inferSchema=True)

# Đặt tên cột thủ công
ratings_df = ratings_df.toDF("user_id", "movie_id", "rating", "timestamp")
ratings_df.show(5)

+-------+--------+------+---------+
|user_id|movie_id|rating|timestamp|
+-------+--------+------+---------+
|      1|    1193|     5|978300760|
|      1|     661|     3|978302109|
|      1|     914|     3|978301968|
|      1|    3408|     4|978300275|
|      1|    2355|     5|978824291|
+-------+--------+------+---------+
only showing top 5 rows



In [6]:
# Sửa thành tên cột đúng
from pyspark.sql.functions import col

als_data = ratings_df.select(
    col("user_id").alias("userId"),  # Đổi tên cột cho ALS
    col("movie_id").alias("movieId"),  # Đổi tên cột cho ALS
    col("rating").cast("float")
).dropna()

print("Total ratings:", als_data.count())
als_data.show(5)

Total ratings: 1000209
+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|   1193|   5.0|
|     1|    661|   3.0|
|     1|    914|   3.0|
|     1|   3408|   4.0|
|     1|   2355|   5.0|
+------+-------+------+
only showing top 5 rows



In [9]:
user_recs = model.recommendForAllUsers(10)

In [10]:
user_recs.show(5, truncate=False)

[Stage 74:====================================================>  (96 + 4) / 100]

+------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|userId|recommendations                                                                                                                                                                        |
+------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1     |[{572, 5.545401}, {557, 4.7572875}, {3233, 4.7557044}, {2129, 4.7107096}, {989, 4.509063}, {527, 4.480734}, {953, 4.4588194}, {318, 4.456668}, {919, 4.4220157}, {3172, 4.410596}]     |
|12    |[{572, 4.8803806}, {557, 4.7985654}, {2309, 4.478616}, {858, 4.4361854}, {989, 4.3725696}, {1420, 4.3672442}, {1221, 4.354765}, {853, 4.3536673}, {3172, 4.3403287}, {296, 4.3009176}] |
|22    |[{557, 4.6074147}, {2309, 4

In [11]:
test_truth = test_data.select("userId", "movieId") \
                      .withColumnRenamed("movieId", "true_movie")

joined = user_recs.join(test_truth, on="userId", how="inner")

In [12]:
def compute_metrics(row):
    true_movie = row["true_movie"]
    recs = [r["movieId"] for r in row["recommendations"]]
    if true_movie in recs:
        rank = recs.index(true_movie) + 1
        hr = 1.0
        ndcg = 1.0 / math.log2(rank + 1)
    else:
        hr = 0.0
        ndcg = 0.0
    return (hr, ndcg)

metrics_rdd = joined.rdd.map(compute_metrics)
metrics_df = metrics_rdd.toDF(["hr", "ndcg"])

result = metrics_df.agg(
    F.avg("hr").alias("HR@10"),
    F.avg("ndcg").alias("NDCG@10")
).collect()[0]

print(f"Hit Ratio @10 = {result['HR@10']:.4f}")
print(f"NDCG @10 = {result['NDCG@10']:.4f}")

25/11/13 12:53:03 ERROR Executor: Exception in task 0.0 in stage 158.0 (TID 399)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4877/1021580721.py", line 7, in compute_metrics
NameError: name 'math' is not defined. Did you forget to import 'math'

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:572)
	at org

Py4JJavaError: An error occurred while calling o226.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 158.0 failed 1 times, most recent failure: Lost task 0.0 in stage 158.0 (TID 399) (172.20.177.130 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4877/1021580721.py", line 7, in compute_metrics


In [14]:
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType, StructType, StructField
import math

# Định nghĩa hàm compute_metrics với import
def compute_metrics(row):
    actual = row['actual']
    predicted = row['recommendations']
    
    # Hit Ratio
    hr = 1 if actual in predicted else 0
    
    # NDCG
    if actual in predicted:
        rank = predicted.index(actual) + 1
        ndcg = 1.0 / math.log2(rank + 1)
    else:
        ndcg = 0.0
        
    return (float(hr), float(ndcg))

# Schema cho kết quả
metrics_schema = StructType([
    StructField("hr", FloatType(), True),
    StructField("ndcg", FloatType(), True)
])

# Tính toán metrics
metrics_rdd = joined.rdd.map(compute_metrics)
metrics_df = spark.createDataFrame(metrics_rdd, metrics_schema)

result = metrics_df.agg(
    F.avg("hr").alias("HR@10"),
    F.avg("ndcg").alias("NDCG@10")
).collect()[0]

print(f"Hit Ratio @10 = {result['HR@10']:.4f}")
print(f"NDCG @10 = {result['NDCG@10']:.4f}")

25/11/13 12:56:03 ERROR Executor: Exception in task 0.0 in stage 175.0 (TID 400)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/sql/types.py", line 2364, in __getitem__
    idx = self.__fields__.index(item)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: 'actual' is not in list

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    re

Py4JJavaError: An error occurred while calling o280.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 175.0 failed 1 times, most recent failure: Lost task 0.0 in stage 175.0 (TID 400) (172.20.177.130 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/sql/types.py", line 2364, in __getitem__
    idx = self.__fields__.index(item)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: 'actual' is not in list

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
 

In [15]:
# Kiểm tra schema của các DataFrame
print("=== test_actual schema ===")
test_actual.printSchema()
test_actual.show(5)

print("=== predictions_df schema ===") 
predictions_df.printSchema()
predictions_df.show(5)

print("=== joined schema ===")
joined.printSchema()
joined.show(5)

=== test_actual schema ===


NameError: name 'test_actual' is not defined

In [16]:
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType, StructType, StructField
from pyspark.ml.evaluation import RegressionEvaluator
import math

# Bước 1: Chuẩn bị dữ liệu train/test
print("=== Chuẩn bị dữ liệu ===")
(train, test) = als_data.randomSplit([0.8, 0.2])
print(f"Train size: {train.count()}")
print(f"Test size: {test.count()}")

# Bước 2: Train ALS model
print("=== Training ALS model ===")
als = ALS(
    userCol="userId",
    itemCol="movieId", 
    ratingCol="rating",
    coldStartStrategy="drop",
    nonnegative=True,
    rank=10,
    maxIter=10
)

model = als.fit(train)

# Bước 3: Tạo predictions
print("=== Tạo predictions ===")
predictions = model.transform(test)
predictions.show(5)

# Bước 4: Đánh giá Regression metrics
print("=== Đánh giá Regression Metrics ===")
evaluator_rmse = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
evaluator_mae = RegressionEvaluator(metricName="mae", labelCol="rating", predictionCol="prediction")

rmse = evaluator_rmse.evaluate(predictions)
mae = evaluator_mae.evaluate(predictions)

print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")

# Bước 5: Tạo recommendations và đánh giá ranking metrics
print("=== Tạo recommendations ===")
# Lấy top 10 recommendations cho mỗi user
user_recs = model.recommendForAllUsers(10)
print("Recommendations schema:")
user_recs.printSchema()
user_recs.show(5, truncate=False)

# Bước 6: Chuẩn bị test_actual (ground truth)
print("=== Chuẩn bị test data ===")
test_actual = test.select(
    "userId",
    "movieId", 
    "rating"
)
print("test_actual schema:")
test_actual.printSchema()
test_actual.show(5)

print("=== predictions_df schema ===")
predictions_df = user_recs
predictions_df.printSchema()
predictions_df.show(5)

# Bước 7: Kiểm tra joined DataFrame
print("=== Kiểm tra joined data ===")
joined = test_actual.join(predictions_df, on="userId")
print("joined schema:")
joined.printSchema()
joined.show(5, truncate=False)

=== Chuẩn bị dữ liệu ===


Train size: 799480


Test size: 200729
=== Training ALS model ===


=== Tạo predictions ===


+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   148|      2|   5.0| 3.6233974|
|   148|      7|   3.0|  3.709041|
|   148|     11|   5.0|  4.061179|
|   148|     15|   5.0| 3.0230236|
|   148|     17|   4.0| 3.7337675|
+------+-------+------+----------+
only showing top 5 rows

=== Đánh giá Regression Metrics ===


...

+------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|userId|recommendations                                                                                                                                                                         |
+------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1     |[{572, 5.600195}, {2776, 4.8494263}, {3172, 4.568014}, {3233, 4.5664563}, {919, 4.5515757}, {3517, 4.544155}, {953, 4.5156007}, {527, 4.494795}, {1035, 4.4469943}, {318, 4.445646}]    |
|12    |[{3172, 4.5885224}, {2494, 4.5723968}, {858, 4.5203404}, {572, 4.502848}, {2309, 4.4913664}, {1221, 4.488925}, {1420, 4.4015493}, {853, 4.3593993}, {1178, 4.3218603}, {1213, 4.311167}]|
|22    |[{2309, 4.294192}, {57

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|    260|   4.0|
|     1|    588|   4.0|
|     1|    720|   3.0|
|     1|    938|   4.0|
|     1|   1270|   5.0|
+------+-------+------+
only showing top 5 rows

=== predictions_df schema ===
root
 |-- userId: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- movieId: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     1|[{572, 5.600195},...|
|    12|[{3172, 4.5885224...|
|    22|[{2309, 4.294192}...|
|    26|[{572, 4.8859205}...|
|    27|[{3542, 4.9551144...|
+------+--------------------+
only showing top 5 rows

=== Kiểm tra joined data ===
joined schema:
root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: float (nullable = true)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- movieId: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



[Stage 524:===================================================>  (96 + 4) / 100]

+------+-------+------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|userId|movieId|rating|recommendations                                                                                                                                                                     |
+------+-------+------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1     |3105   |5.0   |[{572, 5.600195}, {2776, 4.8494263}, {3172, 4.568014}, {3233, 4.5664563}, {919, 4.5515757}, {3517, 4.544155}, {953, 4.5156007}, {527, 4.494795}, {1035, 4.4469943}, {318, 4.445646}]|
|1     |2804   |5.0   |[{572, 5.600195}, {2776, 4.8494263}, {3172, 4.568014}, {3233, 4.5664563}, {919, 4.5515757}, {3517, 4.544155}, {953, 4.5156007}, {527, 4.494795}, {1035, 4.446

In [17]:
# Cách 1: Đánh giá đơn giản với RDD
def compute_metrics(row):
    actual_movie = row['movieId']  # Sử dụng movieId thay vì actual
    predicted_list = [rec['movieId'] for rec in row['recommendations']]
    
    # Hit Ratio
    hr = 1 if actual_movie in predicted_list else 0
    
    # NDCG
    if actual_movie in predicted_list:
        rank = predicted_list.index(actual_movie) + 1
        ndcg = 1.0 / math.log2(rank + 1)
    else:
        ndcg = 0.0
        
    return (float(hr), float(ndcg))

print("=== Tính Ranking Metrics ===")
metrics_rdd = joined.rdd.map(compute_metrics)
metrics_schema = StructType([
    StructField("hr", FloatType(), True),
    StructField("ndcg", FloatType(), True)
])
metrics_df = spark.createDataFrame(metrics_rdd, metrics_schema)

result = metrics_df.agg(
    F.avg("hr").alias("HR@10"),
    F.avg("ndcg").alias("NDCG@10")
).collect()[0]

print(f"Hit Ratio @10 = {result['HR@10']:.4f}")
print(f"NDCG @10 = {result['NDCG@10']:.4f}")

=== Tính Ranking Metrics ===


[Stage 599:==========================================>              (3 + 1) / 4]

Hit Ratio @10 = 0.0071
NDCG @10 = 0.0026


In [18]:
# Cách 2: Sử dụng Pandas UDF cho hiệu suất tốt hơn
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import FloatType
import pandas as pd

@pandas_udf(FloatType())
def calculate_hr_udf(actual_movies: pd.Series, recommendations: pd.Series) -> pd.Series:
    results = []
    for actual, recs in zip(actual_movies, recommendations):
        predicted_list = [r['movieId'] for r in recs]
        hr = 1.0 if actual in predicted_list else 0.0
        results.append(hr)
    return pd.Series(results)

@pandas_udf(FloatType())
def calculate_ndcg_udf(actual_movies: pd.Series, recommendations: pd.Series) -> pd.Series:
    results = []
    for actual, recs in zip(actual_movies, recommendations):
        predicted_list = [r['movieId'] for r in recs]
        if actual in predicted_list:
            rank = predicted_list.index(actual) + 1
            ndcg = 1.0 / math.log2(rank + 1)
        else:
            ndcg = 0.0
        results.append(ndcg)
    return pd.Series(results)

# Tính metrics với Pandas UDF
results_df = joined.withColumn("hr", calculate_hr_udf(F.col("movieId"), F.col("recommendations"))) \
                  .withColumn("ndcg", calculate_ndcg_udf(F.col("movieId"), F.col("recommendations")))

final_result = results_df.agg(
    F.avg("hr").alias("HR@10"),
    F.avg("ndcg").alias("NDCG@10")
).collect()[0]

print(f"Hit Ratio @10 = {final_result['HR@10']:.4f}")
print(f"NDCG @10 = {final_result['NDCG@10']:.4f}")

ImportError: PyArrow >= 1.0.0 must be installed; however, it was not found.

In [ ]:
# =============================================
# 4️⃣ Tính Top 10 movies cho tất cả users
# =============================================
user_recs = model.recommendForAllUsers(10)
user_recs.show(5, truncate=False)